In [17]:
new_path = r"datasets\\RecipeDB.Recipes.json"

In [18]:
import pandas as pd

df = pd.read_json(new_path)
df.head()

,_id,title,directions,NER
0,68ed69731541b6b22612a1c8,Pecan Pie,"[Set oven to 350°., Beat first 5 ingredients. ...","[eggs, sugar, corn syrup, vanilla, butter, pec..."
1,68ed69731541b6b22612a1c9,Cheese Biscuits,"[Soften butter and cheese., Gradually add flou...","[margarine, cheese, flour, paprika]"
2,68ed69731541b6b22612a1ca,Breaded Ranch Chicken,"[In a shallow bowl, combine cornflakes, Parmes...","[cornflakes, Parmesan cheese, salad dressing, ..."
3,68ed69731541b6b22612a1cb,Mamma Jo'S Calico Bean Hot Dish,"[In a large skillet, fry together bacon, hambu...","[bacon, hamburger, onion, brown sugar, mustard..."
4,68ed69731541b6b22612a1cc,Venison Meat Loaf,"[Combine all ingredients thoroughly., Place in...","[ground venison, ground pork, onion, grated ca..."


In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import lancedb

# GPU for recipe embeddings
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
recipe_embedder = SentenceTransformer(
    "sentence-transformers/all-MiniLM-L6-v2", device=device
)

db = lancedb.connect("./lance_db")

cuda


In [22]:
recipe_texts = df["NER"].apply(lambda x: " ".join(x)).tolist()
recipe_embeddings = recipe_embedder.encode(
    recipe_texts, convert_to_tensor=True, device=device
)

In [ ]:
import numpy as np

# recipe_embeddings is a PyTorch tensor on GPU so converting to flipping numpy
recipe_embeddings_np = recipe_embeddings.cpu().numpy().astype("float32")
df["embedding"] = list(recipe_embeddings_np)

In [24]:
recipe_table = db.create_table(name="recipes", data=df, mode="overwrite")

In [26]:
recipe_table.create_index(
    vector_column_name="embedding",
    index_type="IVF_PQ",
    metric="cosine",
    num_partitions=256,
    num_sub_vectors=64,
)